In [1]:
# Importação de bibliotecas
import pandas as pd

In [2]:
# Objetos
fonte = 'sidra'
dataset_name = 'tab1092_bovino'

raiz = '/home/eduardo/repositorios/IS_Agro/databases'

### Bronze

In [3]:
(pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n3/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
 .to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_uf.json')
 )

(pd.read_json('https://apisidra.ibge.gov.br/values/t/1092/n1/all/v/285/p/last%2094/c12716/115236/c18/992/c12529/118225')
 .to_json(f'{raiz}/bronze_{fonte}_{dataset_name}_BR.json')
 )

### Prata

In [4]:
(pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_uf.json')
 .filter(['D1C', 'D3C', 'V'])
 .drop(0, axis = 0)
 .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'data': lambda df: pd.to_datetime(df['ano'], format='%Y'), # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int)
            })
 .dropna(subset='peso_kg')
 .groupby(['uf', 'data'])
 .agg({'peso_kg': 'sum'})
 .to_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet')
 )

In [5]:
df_br = (pd.read_json(f'{raiz}/bronze_{fonte}_{dataset_name}_BR.json')
 .filter(['D1C', 'D3C', 'V'])
 .drop(0, axis = 0)
 .assign(**{'ano': lambda tri: tri['D3C'].str.extract('(\\d{4})').astype(int),
            'data': lambda df: pd.to_datetime(df['ano'], format='%Y'), # Coluna "data" em datetime baseado na ISO 8601, convertido da coluna "D3C"
            'peso_kg': lambda df: pd.to_numeric(df['V'], errors='coerce'),
            'uf': lambda df: df['D1C'].astype(int)
            })
 .groupby(['uf', 'data'])
 .agg({'peso_kg': 'sum'})
 )

### Ouro

Peso total de carcaças dos bovinos abatidos (kg)

Aproveitamento da carcaça: 50%

Cálculo: ((Carcaça Bovina/aproveitamento%) * (Parâmetro_x%))/1000

UF e Brasil = t

NPK na forma de Carne bovina retirada do solo via animal encaminhado para abate (t)

In [6]:
aproveitamento = 50/100

parametro_N = 3/100
parametro_P = 0.8/100
parametro_K = 0.2/100

In [9]:
df = (pd.concat([pd.read_parquet(f'{raiz}/prata_{fonte}_{dataset_name}.parquet'), df_br])
      .assign(**{
          'N': lambda df: ((df['peso_kg']/aproveitamento)*parametro_N)/1000,
          'P': lambda df: ((df['peso_kg']/aproveitamento)*parametro_P)/1000,
          'K': lambda df: ((df['peso_kg']/aproveitamento)*parametro_K)/1000,
          'fonte': 'Carcaça Bovina'
          })
      .reset_index()
      .melt(id_vars=['uf', 'data', 'fonte'],
            value_vars=['N', 'P', 'K'],
            var_name='elemento',
            value_name='valor')
      .convert_dtypes()
      .set_index(['uf', 'data', 'fonte'])
      )

In [12]:
df = (
    df
    .rename_axis(index={
        'uf':'state',
        'data': 'date',
        'fonte': 'label'
    })
    .rename(columns={
        'elemento':'nutrient',
        'valor':'value'
    })
)

df.to_csv(f'{raiz}/ouro_npk_carcaca_bovina.csv')